## Install libraries

In [ ]:
# Detectron2
# might take couple of minutes
!python -m pip install pyyaml==5.1
!git clone https://github.com/facebookresearch/detectron2.git
# if raises error on Windows --> insert "-DWITH_CUDA" to the extra_compile_args["nvcc"] on setup.py
!python -m pip install -e detectron2

In [ ]:
# YOLO
!git clone https://github.com/ultralytics/yolov5.git 
!pip install ultralytics
!pip install -r yolov5/requirements.txt

In [ ]:
# DETR
!git clone https://github.com/facebookresearch/detr.git

In [ ]:
# For the generaiton of COCO Format
!git clone https://github.com/yukkyo/voc2coco.git

In [ ]:
import os
import shutil
import torch
import torchvision
import detectron2


from utils_environment import *

# Dataset URL
URL = "https://osf.io/download/3qx7m/"
# TRUE IF ON COLAB 
COLAB = False

## Setup env

In [ ]:
# mount drive if colab else make compatibility
if COLAB:
    shutil.rmtree("sample_data")
    
if COLAB:
    from google.colab import files, drive
    drive.mount("content/")
else:
    os.makedirs(os.path.join("content", "MyDrive"), exist_ok=True)

In [ ]:
# check cuda, torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
TORCHVISION_VERSION = ".".join(torch.__version__.split(".")[:2])
TORCHVISION_CUDA_VERSION = torch.__version__.split("+")[-1]
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)
print("torchvision: ", TORCHVISION_VERSION, "; cuda: ", TORCHVISION_CUDA_VERSION)

In [ ]:
# download, decompress the data
ZIP_NAME = 'CGHD-1152.zip'

if COLAB:
    !wget https://osf.io/download/3qx7m/ --verbose
    !unzip index.html > /dev/null
else:
    install_dataset(ZIP_NAME, URL)
    unzip(ZIP_NAME)

In [ ]:
# Generate the files for yolo format
enumerate_classes()
# load to inspect (move to drive manually beforehand)
class_name_to_id_mapping, class_names = load_classes_from_drive()
generate_yolo_yaml_file()

# makedirs for YOLO Dataset
os.makedirs(os.path.join("CGHD-1152-YOLO", "labels", "test"), exist_ok=True)
os.makedirs(os.path.join("CGHD-1152-YOLO", "labels", "val"), exist_ok=True)
os.makedirs(os.path.join("CGHD-1152-YOLO", "labels", "train"), exist_ok=True)
os.makedirs(os.path.join("CGHD-1152-YOLO", "images", "test"), exist_ok=True)
os.makedirs(os.path.join("CGHD-1152-YOLO", "images", "val"), exist_ok=True)
os.makedirs(os.path.join("CGHD-1152-YOLO", "images", "train"), exist_ok=True)

generate_annotations_yolo_format()
populate_yolo_folders()

In [ ]:
setup_for_voc2coco(class_name_to_id_mapping)

# Generate Json Annotations
!python ./voc2coco/voc2coco.py --ann_dir "Annotations/" --ann_ids "dataset_ids/train.txt" --labels "labels.txt" --output "train.json"
!python ./voc2coco/voc2coco.py --ann_dir "Annotations/" --ann_ids "dataset_ids/val.txt" --labels "labels.txt" --output "val.json"
!python ./voc2coco/voc2coco.py --ann_dir "Annotations/" --ann_ids "dataset_ids/test.txt" --labels "labels.txt" --output "test.json"

# Optional
cleanup_voc2coco()

# Move to Drive for later use
assert shutil.copy("train.json", os.path.join("content", "MyDrive", "CGHD_COCO_Format"))
assert shutil.copy("test.json", os.path.join("content", "MyDrive", "CGHD_COCO_Format"))
assert shutil.copy("val.json", os.path.join("content", "MyDrive", "CGHD_COCO_Format"))